In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from astropy import units as u
from astropy.coordinates import SkyCoord
import SAGA
from SAGA import ObjectCuts as C
from dotenv import load_dotenv
import matplotlib.colors as mcolors

load_dotenv()

In [ ]:
saga_dir = os.getenv('SAGA_DIR')
saga_db = os.getenv('SAGA_DB')

print(saga_db)

In [ ]:
# Load SAGA database
saga = SAGA.QuickStart(local_dir=saga_dir)
 
saga.database["combined_base"].remote.path = saga_db
saga.database["combined_base"].download(overwrite=True)

In [ ]:
# Load catalogs
base = saga.object_catalog.load_combined_base_catalog()
base = saga.host_catalog.construct_host_query("paper3").filter(base)

sats = C.is_sat.filter(base)
hosts = C.is_host.filter(base)

df = pd.DataFrame(dict(sats))
print(list(df.columns))
host_df = pd.DataFrame(dict(hosts)) 
host_df[['HOSTID','SGA_ID','radius']].sort_values('HOSTID')


# returns the distance in Mpc corresponding to cosmological redshift z and a given value of h (defaults to cosmosim 0.6777)
def get_d_from_z(z, h = 0.6777):
    H_0 = 100*h # Hubble constant in km/s/Mpc
    return z*299792/H_0 # speed of light in km/s

host_df['distance'] = get_d_from_z(host_df['HOST_ZCOSMO'])
host_df['HOST_RVIR_ANGULAR_DEG'] = 0.3/host_df['distance']*360/(2*np.pi) # using the small-angle approximation and 300 kpc (0.3 Mpc) rvir assumption 

In [ ]:
host_list = df["HOSTID"].unique()

In [ ]:
custom_style = {
    "figure.facecolor": "#000000",
    "axes.facecolor": "#000000",
    "savefig.facecolor": "#000000", 
    "grid.color": "#000000",
    # "grid.color": "#ffe599",
    # "grid.alpha": 0.1,
    "text.color": "#ffe599",
    "axes.labelcolor": "#ffe599",
    "xtick.color": "#ffe599",
    "ytick.color": "#ffe599",
    "grid.linestyle": "-",
    "lines.solid_capstyle": "round"
}

sns.set_style("ticks", custom_style)

In [ ]:
# Plot of all satellites, color-grouped by host
g = sns.relplot(x="RA", y="DEC", hue="HOSTID", palette="viridis", data=df, facet_kws=dict(legend_out=False))
g._legend.remove()

In [ ]:
df['LAT_ROTATED'] = SkyCoord(ra=df.RA*u.degree, dec=df.DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).lat.deg
df['LON_ROTATED'] = SkyCoord(ra=df.RA*u.degree, dec=df.DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).lon.deg

# acutally ra and dec
df['HOST_LAT_ROTATED'] = SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).lat.deg
df['HOST_LON_ROTATED'] = SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').transform_to(SkyCoord(ra=df.HOST_RA*u.degree, dec=df.HOST_DEC*u.degree, frame='icrs').skyoffset_frame()).lon.deg

In [ ]:
host_ids = ['pgc66566', 'nsa16235','nsa135739']
common_names = ['NGC 7051', 'UGC 04906','NGC 2778']
prominences = [212.765957, 500.000000, 588.235294]
just3hosts_df = df[df.HOSTID.isin(host_ids)]


host_ids_2 = ['pgc66566', 'nsa16235','nsa135739', 'nsa129237', 'nsa133115', 'pgc53630']
common_names_2 = ['NGC 7051', 'UGC 04906','NGC 2778', 'UGC 00903', 'NGC 1199', 'NGC 5812']
prominences_2 = [212.765957, 500.000000, 588.235294, 1.085658, 1.974334, 8.741259]
just3hosts_df_2 = df[df.HOSTID.isin(host_ids_2)]

In [ ]:
for host in host_ids:
    host_rmag = host_df[host_df['HOSTID'] == host].head(1)[['r_mag']].values[0][0]
    host_radius = host_df[host_df['HOSTID'] == host].head(1)[['radius']].values[0][0]
    HOST_RVIR_ANGULAR_DEG = host_df[host_df['HOSTID'] == host].head(1)[['HOST_RVIR_ANGULAR_DEG']].values[0][0]

    just3hosts_df.loc[host] = np.zeros(len(just3hosts_df.columns))
    just3hosts_df.loc[[host],['r_mag']] = host_rmag
    just3hosts_df.loc[[host],['radius']] = host_radius
    just3hosts_df.loc[[host],['HOST_RVIR_ANGULAR_DEG']] = HOST_RVIR_ANGULAR_DEG
    just3hosts_df.loc[[host],['HOSTID']] = host

In [ ]:
for host in host_ids_2:
    host_rmag = host_df[host_df['HOSTID'] == host].head(1)[['r_mag']].values[0][0]
    host_radius = host_df[host_df['HOSTID'] == host].head(1)[['radius']].values[0][0]
    HOST_RVIR_ANGULAR_DEG = host_df[host_df['HOSTID'] == host].head(1)[['HOST_RVIR_ANGULAR_DEG']].values[0][0]

    just3hosts_df_2.loc[host] = np.zeros(len(just3hosts_df_2.columns))
    just3hosts_df_2.loc[[host],['r_mag']] = host_rmag
    just3hosts_df_2.loc[[host],['radius']] = host_radius
    just3hosts_df_2.loc[[host],['HOST_RVIR_ANGULAR_DEG']] = HOST_RVIR_ANGULAR_DEG
    just3hosts_df_2.loc[[host],['HOSTID']] = host

In [ ]:
# Rotated RA/DEC plots (host @ 0,0), including host points
pgc66566_df = just3hosts_df[just3hosts_df['HOSTID'] == host_ids[0]]
nsa16235_df = just3hosts_df[just3hosts_df['HOSTID'] == host_ids[1]]
nsa135739_df = just3hosts_df[just3hosts_df['HOSTID'] == host_ids[2]]
dfs = [pgc66566_df, nsa16235_df, nsa135739_df]
virial_radii = [0.52, 0.51, 0.56] # GET THESE FROM DATAFRAME - NEED TO CONVERT TO DEGREES

fig, axs = plt.subplots(ncols=3, sharey=True, sharex=True)
fig.set_size_inches(11, 3, forward=True)

for i, ax in enumerate(axs):
    radius = just3hosts_df.loc[[host_ids[i]],['HOST_RVIR_ANGULAR_DEG']].values[0][0]
    ax.add_patch(plt.Circle(xy=(0, 0), radius=radius, color='gray', fill=False, linewidth=2, linestyle='--'))

    sp = ax.scatter(dfs[i].LON_ROTATED, dfs[i].LAT_ROTATED, c=dfs[i].r_mag, s=65, linewidth=0.7, edgecolor='white', cmap=plt.cm.get_cmap('viridis').reversed())
    
    if i == 0: ax.set_ylabel('Dec (deg)', fontsize=12)
    if i == 1: ax.set_xlabel('RA (deg)', fontsize=12)
    ax.set_xlim(-0.6, 0.6)
    ax.set_ylim(-0.6, 0.6)
    ax.set_title(common_names[i]+" ($\mathcal{P}_e$: "+str(round(prominences[i]))+')')

cbar = fig.colorbar(sp, ax=axs.ravel().tolist(), shrink=0.85)
cbar.ax.set_title('$m_r$')

plt.subplots_adjust(wspace=0, hspace=0, left=0.1, right=0.75)
fig.savefig('figures/SpatialPlots.png', dpi=1000)

In [ ]:
# Rotated RA/DEC plots (host @ 0,0), including host points
dfs = []
virial_radii = []

for host in host_ids_2:
    df = just3hosts_df_2[just3hosts_df_2['HOSTID'] == host].copy()
    dfs.append(df)
    virial_radius = df.loc[host].HOST_RVIR_ANGULAR_DEG
    virial_radii.append(virial_radius)


fig, axs = plt.subplots(ncols=3, nrows=2, sharey=True, sharex=True)
fig.set_size_inches(11, 6, forward=True)

for j, axis_row in enumerate(axs):
    for i, ax in enumerate(axis_row):
        if j == 1: i += 3
        radius = just3hosts_df_2.loc[[host_ids_2[i]],['HOST_RVIR_ANGULAR_DEG']].values[0][0]
        ax.add_patch(plt.Circle(xy=(0, 0), radius=radius, color='gray', fill=False, linewidth=2, linestyle='--'))

        sp = ax.scatter(dfs[i].LON_ROTATED, dfs[i].LAT_ROTATED, c=dfs[i].r_mag, s=65, linewidth=0.7, edgecolor='white', cmap=plt.cm.get_cmap('viridis').reversed())
        
        if (j == 0 and i == 0): ax.set_ylabel('$\Delta$ Dec (deg)', fontsize=12)
        if i == 4: ax.set_xlabel('$\Delta$ RA (deg)', fontsize=12)
        ax.set_xlim(-0.65, 0.65)
        ax.set_ylim(-0.65, 0.65)
        ax.set_title(common_names_2[i]+" ($\mathcal{P}_e$: "+str(round(prominences_2[i],1))+')')

cbar = fig.colorbar(sp, ax=axs.ravel().tolist(), shrink=0.85)
cbar.ax.set_title('$m_r$')

plt.subplots_adjust(wspace=0, hspace=0.05, left=0.1, right=0.75)
fig.savefig('figures/SpatialPlots.png', dpi=1000)

In [ ]:
pgc66566_df = just3hosts_df[just3hosts_df['HOSTID'] == host_ids[0]]
nsa16235_df = just3hosts_df[just3hosts_df['HOSTID'] == host_ids[1]]
nsa135739_df = just3hosts_df[just3hosts_df['HOSTID'] == host_ids[2]]
prominences = [212.765957, 500.000000, 588.235294]
dfs = [pgc66566_df, nsa16235_df, nsa135739_df]
virial_radii = [0.52, 0.51, 0.56] # GET THESE FROM DATAFRAME - NEED TO CONVERT TO DEGREES

fig, axs = plt.subplots(ncols=3, sharey=True, sharex=True)
fig.set_size_inches(11, 3, forward=True)

for i, ax in enumerate(axs):
    radius = just3hosts_df.loc[[host_ids[i]],['HOST_RVIR_ANGULAR_DEG']].values[0][0]
    ax.add_patch(plt.Circle(xy=(0, 0), radius=radius, color='gray', fill=False, linewidth=2, linestyle='--'))

    sp = ax.scatter(dfs[i].LON_ROTATED, dfs[i].LAT_ROTATED, c=dfs[i].r_mag, s=65, linewidth=0.7, edgecolor='white', cmap=plt.cm.get_cmap('viridis').reversed())
    
    # ax.set_aspect('equal')
    if i == 0: ax.set_ylabel('Latitude (deg)', fontsize=12)
    if i == 1: ax.set_xlabel('Longitude (deg)', fontsize=12)
    ax.set_xlim(-0.6, 0.6)
    ax.set_ylim(-0.6, 0.6)
    ax.set_title(common_names[i]+" ($\mathcal{P}_e$: "+str(round(prominences[i]))+')')

cbar = fig.colorbar(sp, ax=axs.ravel().tolist(), shrink=0.85)
cbar.ax.set_title('$m_r$')

plt.subplots_adjust(wspace=0, hspace=0, left=0.1, right=0.75)
fig.savefig('figures/SpatialPlots.png', dpi=1000)

<h4>Testing rotation accuracy</h4>
Ensure that diff(Host RA,0) and diff(Host DEC,0) are no greater than inherent error due to float calculations 

In [ ]:
df[(df["HOST_LON_ROTATED"] != 0) | (df["HOST_LAT_ROTATED"] != 0)]

In [ ]:
# Check to see if host position after rotation is non-zero (above float calc error)
lon_err_max = df[df["HOST_LON_ROTATED"] != 0].HOST_LON_ROTATED.max()
lon_err_min = df[df["HOST_LON_ROTATED"] != 0].HOST_LON_ROTATED.min()
lat_err_max = df[df["HOST_LAT_ROTATED"] != 0].HOST_LAT_ROTATED.max()
lat_err_min = df[df["HOST_LAT_ROTATED"] != 0].HOST_LAT_ROTATED.min()
print("Error: lat min - {}, \t lat max - {}, \t lon min - {}, \t lon max - {}".format(lat_err_min, lat_err_max, lon_err_min, lon_err_max))

In [ ]:
# just to get plot limits
lon_max = df[df["LON_ROTATED"] != 0].LON_ROTATED.max()
lon_min = df[df["LON_ROTATED"] != 0].LON_ROTATED.min()
lat_max = df[df["LAT_ROTATED"] != 0].LAT_ROTATED.max()
lat_min = df[df["LAT_ROTATED"] != 0].LAT_ROTATED.min()
print("lat min - {}, \t lat max - {}, \t lon min - {}, \t lon max - {}".format(lat_min, lat_max, lon_min, lon_max))